In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 22.5 MB/s 


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random
import numpy as np

In [ ]:
cacheSize = 2           #Number of lines per cache
lineSize = 2            #Number of memory objects in each cache line
accessSeqSize = 5      #Number of memory objects being accessed
memoryObjectCount = 10  #Number of distinct memory objects being accessed
setCount = 8            #Number of Cache Sets

In [ ]:
accessSeq = []
for i in range (0,accessSeqSize):
  n = random.randint(0,memoryObjectCount-1)
  accessSeq.append(n)

In [ ]:
print(accessSeq)

[3, 6, 1, 0, 8]


In [ ]:
m = gp.Model('LRU')

Restricted license - for non-production use only - expires 2024-10-28


In [ ]:
x = m.addVars(setCount,memoryObjectCount,memoryObjectCount,vtype=GRB.BINARY)

In [ ]:
age = m.addVars(setCount,memoryObjectCount,accessSeqSize+1,vtype=GRB.INTEGER) #variables to store ages of all accessed objects over time

In [ ]:
multiplicationMatrix = m.addVars(setCount,memoryObjectCount,accessSeqSize,vtype=GRB.INTEGER) 

In [ ]:
neg = m.addVars(setCount,memoryObjectCount,accessSeqSize,vtype = GRB.BINARY)

In [ ]:
hit = m.addVars(setCount,accessSeqSize,1,vtype = GRB.BINARY)

In [ ]:
accessedObjectAge = m.addVars(accessSeqSize,1,vtype=GRB.INTEGER)

In [ ]:
hitCountHelper = m.addVars(setCount,accessSeqSize,1,vtype=GRB.INTEGER)

In [ ]:
multidimHit = m.addVars(setCount,accessSeqSize,vtype = GRB.BINARY)

For the mapping matrix 'x' where the rows represent the memory objects and columns represent the group numbers, the following inequality should hold:

$\sum_{i=0}^{setCount-1}\sum_{k=0}^{memoryObjectCount-1} x_{i,j,k} = 1$  $ ∀ j \in [0,memoryObjectCount-1]$




In [ ]:
uniqueLineConstraint = m.addConstrs(((gp.quicksum(x[i,j,k] for i in range (0,setCount) for k in range (0,memoryObjectCount)) == 1) for j in range (0,memoryObjectCount)), name='uniqueLineConstraint')

$\sum_{j=0}^{memoryObjectCount-1} x_{i,j,k} <= lineSize$ $ ∀ i \in [0,setCount-1] k \in [0,memoryObjectCount-1]$

In [ ]:
cacheSizeConstraint = m.addConstrs(((gp.quicksum(x[i,j,k] for j in range (0,memoryObjectCount)) <= lineSize) for i in range (0,setCount) for k in range (0,memoryObjectCount)), name='cacheSizeConstraint')

$multiplicationMatrix_{i,j,k} = $  $x_{i,accessSeq_{k},j}$ $\forall i \in [0,setCount-1],\forall j \in [0,memoryObjectCount-1]$, $\forall k \in [0,accessSeqSize-1]$

In [ ]:
multiplicationMatrixConstraint = m.addConstrs((multiplicationMatrix[i,j,k] == x[i,accessSeq[k],j] for i in range (0,setCount) for j in range (0,memoryObjectCount) for k in range (0,accessSeqSize)), name='multiplicationMatrixConstraint')

$accessedObjectAge_{k} = \sum_{j=0}^{memoryObjectCount-1} \sum_{i=0}^{setCount-1}(multiplicationMatrix_{i,j,k}*age_{i,j,k})$ $\forall k \in [0,accessSeqSize-1]$

In [ ]:
accessedObjectAgeConstraint = m.addConstrs((accessedObjectAge[k,0] == gp.quicksum(multiplicationMatrix[i,j,k]*age[i,j,k] for i in range (0,setCount) for j in range (0,memoryObjectCount)) for k in range (0,accessSeqSize)), name = 'accessedObjectAgeConstraint')

$age_{i,j,k} = (accessSequenceSize + 2)$ $\forall i \in [0,setCount-1],\forall j \in [0,memoryObjectCount-1],\forall k \in \{0\}$ 

$age_{i,j,k} = (1-multiplicationMatrix_{i,j,k-1})*(age_{i,j,k-1} + 1)$ $\forall i \in [0,setCount-1]$ $ \forall j \in [0,memoryObjectCount-1]$, $\forall k \in [1,accessSeqSize]$ 

In [ ]:
#Store age of each group at every access-point
ageBoundaryConstraint = m.addConstrs((age[i,j,0] == accessSeqSize+2 for i in range(0,setCount) for j in range (0,memoryObjectCount)),name = 'ageBoundaryConstraint')
ageConstraint = m.addConstrs((age[i,j,k] == (1-multiplicationMatrix[i,j,k-1])*(age[i,j,k-1]+1) for i in range (0,setCount) for j in range (0,memoryObjectCount) for k in range (1,accessSeqSize+1)),name = 'ageConstraint')

$neg_{i,j,k} = (age_{i,j,k} <= accessedObjectAge_{k}-1)$ $\forall i \in [0,setCount-1]$, $\forall j \in [0,memoryObjectCount-1],$, $\forall k \in [0,accessSeqSize-1]$

In [ ]:
for i in range (0,setCount):
  for j in range (0,memoryObjectCount):
    for k in range (0,accessSeqSize):
      m.addGenConstrIndicator(neg[i,j,k],  0,age[i,j,k] >=  accessedObjectAge[k,0]+1)
      m.addGenConstrIndicator(neg[i,j,k],  1,age[i,j,k] <=  accessedObjectAge[k,0])

$hitCountHelper_{i,k} = \sum_{j=0}^{memoryObjectCount-1}neg_{i,j,k}$ $\forall i \in [0,setCount-1],$ $ \forall k \in [0,accessSeqSize-1]$ 

In [ ]:
m.addConstrs(hitCountHelper[i,k,0] == gp.quicksum(neg[i,j,k] for j in range (0,memoryObjectCount)) for i in range (0,setCount) for k in range (0,accessSeqSize))

$hit_{i,j} = (hitCountHelper_{i,j} <= cacheSize-1)$ $\forall i \in [0,setCount-1]$ $∀ j \in [0,accessSeqSize-1]$

In [ ]:
for i in range (0,setCount):
  for j in range (0,accessSeqSize):
      m.addGenConstrIndicator(hit[i,j,0],  0,hitCountHelper[i,j,0] >=  cacheSize+1)
      m.addGenConstrIndicator(hit[i,j,0],  1,hitCountHelper[i,j,0] <=  cacheSize)

$multidimHit_{i,j} = (\sum_{k=0}^{memoryObjectCount-1}multiplicationMatrix_{i,k,j})*hit_{i,j} \forall i \in [0,setCount-1], j \in [0,accessSeqSize-1]$

In [ ]:
m.addConstrs(multidimHit[i,j] == gp.quicksum(multiplicationMatrix[i,k,j] for k in range (0,memoryObjectCount))*hit[i,j,0] for i in range (0,setCount) for j in range (0,accessSeqSize))

$Objective$ $Function: \sum_{i=0}^{setCount-1}\sum_{j=0}^{accessSeqSize-1}multidimHit_{i,j}$

In [ ]:
m.setObjective(gp.quicksum(multidimHit[i,j] for i in range (0,setCount) for j in range (0,accessSeqSize)), GRB.MAXIMIZE)

In [ ]:
m.write('LRU.lp')

In [ ]:
m.optimize()